# RISC-V Processor FPGA Verification - Physical Button Testing
## Hardware Button Controller Mode

**Hardware topology:**
```
  PS (ARM / Jupyter)                 PL (FPGA fabric)
  ─────────────────    AXI-Lite      ────────────────────────────────────
  MMIO(iram) ──────────────────────► axi_bram_ctrl_1  → Instruction RAM
  MMIO(dram) ──────────────────────► axi_bram_ctrl_0  → Data RAM
                                                           │
                                      Button Controller ───┤
                                            │              │
                                       BTN0-BTN3      RISC-V Core ──► done_flag_0 (LED0)
```

**Testing Mode:**
This notebook is designed for **physical button control** using the button controller bitstream.
- Python loads program and data to BRAM
- Physical buttons (BTN0-BTN3) control processor execution
- Python monitors completion and verifies results

**LED Configuration:**
- **LED0 (R14)**: `done_flag_0` from processor (completion indicator)
- **LED1 (P14)**: Halted status
- **LED2 (N16)**: Single-step mode
- **LED3 (M14)**: Force done flag

The program is a **fully-unrolled bubble sort** with no `jal`/`jalr` instructions.

## 1. Imports & Test Parameters

In [1]:
import struct
import time
import random
import threading
from typing import List, Tuple

# ─── Test parameters ─────────────────────────────────────────────────────────
NUM_ELEMENTS_TO_SORT = 32
PRNG_SEED_VALUE = 42
MAX_EXECUTION_TIMEOUT_SECONDS = 10.0

# ─── Memory-map constants ────────────────────────────────────────────────────
DRAM_SORT_ARRAY_BASE_ADDR = 0x000
DRAM_COMPLETION_FLAG_ADDR = 0x100
COMPLETION_MAGIC_VALUE = 0xDEADBEAF

SIGNED_INT32_MINIMUM_VALUE = -100
SIGNED_INT32_MAXIMUM_VALUE = 100

print('[OK] Imports loaded')
print(f'  Array size : {NUM_ELEMENTS_TO_SORT} elements')
print(f'  Sentinel   : 0x{COMPLETION_MAGIC_VALUE:08X} @ dram+0x{DRAM_COMPLETION_FLAG_ADDR:03X}')

[OK] Imports loaded
  Array size : 32 elements
  Sentinel   : 0xDEADBEAF @ dram+0x100


## 2. Load Overlay & Map Hardware

In [2]:
from pynq import Overlay, MMIO

BITSTREAM_FILENAME = 'design_1_wrapper.bit'

print('Loading overlay...')
fpga_overlay = Overlay(BITSTREAM_FILENAME)
print('[OK] Overlay loaded')

# ───BRAM controllers ────────────────────────────────────────────────────────
iram_interface_info = fpga_overlay.mem_dict['axi_bram_ctrl_1']
dram_interface_info = fpga_overlay.mem_dict['axi_bram_ctrl_0']

instruction_memory = MMIO(iram_interface_info['phys_addr'], iram_interface_info['addr_range'])
data_memory = MMIO(dram_interface_info['phys_addr'], dram_interface_info['addr_range'])

print(f'[OK] Hardware mapped')
print(f'  iram @ 0x{iram_interface_info["phys_addr"]:08X}')
print(f'  dram @ 0x{dram_interface_info["phys_addr"]:08X}')
print()
print('Note: Button controller in bitstream controls processor reset')
print('      (No GPIO control used in this configuration)')

Loading overlay...


[OK] Overlay loaded
[OK] Hardware mapped
  iram @ 0x42000000
  dram @ 0x40000000

Note: Button controller in bitstream controls processor reset
      (No GPIO control used in this configuration)


## 3. Machine-Code Program Generator

In [3]:
def _extract_bits(x, bits): 
    return x & ((1 << bits) - 1)

# ── RV32I instruction encoders ───────────────────────────────────────────────
def encode_beq(rs1, rs2, imm):
    assert imm % 2 == 0, 'branch offset must be even'
    imm = _extract_bits(imm, 13)
    return (((imm>>12)&1)<<31) | (((imm>>5)&0x3F)<<25) | (_extract_bits(rs2,5)<<20) | \
           (_extract_bits(rs1,5)<<15) | (0b000<<12) | (((imm>>1)&0xF)<<8) | (((imm>>11)&1)<<7) | 0x63

def encode_slt(rd, rs1, rs2):
    return (0b0000000<<25) | (_extract_bits(rs2,5)<<20) | (_extract_bits(rs1,5)<<15) | \
           (0b010<<12) | (_extract_bits(rd,5)<<7) | 0x33

def encode_addi(rd, rs1, imm):
    return (_extract_bits(imm,12)<<20) | (_extract_bits(rs1,5)<<15) | \
           (0b000<<12) | (_extract_bits(rd,5)<<7) | 0x13

def encode_lw(rd, rs1, imm):
    return (_extract_bits(imm,12)<<20) | (_extract_bits(rs1,5)<<15) | \
           (0b010<<12) | (_extract_bits(rd,5)<<7) | 0x03

def encode_lui(rd, imm):
    return (_extract_bits(imm,20)<<12) | (_extract_bits(rd,5)<<7) | 0x37

def encode_sw(rs2, rs1, imm):
    imm = _extract_bits(imm, 12)
    return (((imm>>5)&0x7F)<<25) | (_extract_bits(rs2,5)<<20) | (_extract_bits(rs1,5)<<15) | \
           (0b010<<12) | ((imm&0x1F)<<7) | 0x23

# ── Fully-unrolled bubble sort generator ─────────────────────────────────────
def generate_unrolled_bubble_sort_program() -> Tuple[List[int], List[str]]:
    """
    Generates fully-unrolled bubble sort (no jal/jalr).
    
    Register allocation:
      x0  = hardwired zero (base address pointer)
      x7  = arr[j]
      x8  = arr[j+1]
      x9  = comparison result
      x12 = sentinel builder
    """
    x0, x7, x8, x9, x12 = 0, 7, 8, 9, 12
    instructions = []
    assembly = []

    # Generate 496 compare-swap blocks (31 passes × descending inner loops)
    for pass_num in range(31):
        for j in range(31 - pass_num):
            off = 4 * j
            instructions.append(encode_lw(x7, x0, off));      assembly.append(f'lw   x7, {off}(x0)')
            instructions.append(encode_lw(x8, x0, off+4));    assembly.append(f'lw   x8, {off+4}(x0)')
            instructions.append(encode_slt(x9, x8, x7));      assembly.append(f'slt  x9, x8, x7')
            instructions.append(encode_beq(x9, x0, 12));      assembly.append(f'beq  x9, x0, +12')
            instructions.append(encode_sw(x8, x0, off));      assembly.append(f'sw   x8, {off}(x0)')
            instructions.append(encode_sw(x7, x0, off+4));    assembly.append(f'sw   x7, {off+4}(x0)')

    # Epilogue: write 0xDEADBEEF to dram[0x100], then halt
    instructions.append(encode_lui(x12, 0xDEADC));            assembly.append('lui  x12, 0xDEADC')
    instructions.append(encode_addi(x12, x12, -337));         assembly.append('addi x12, x12, -337')
    instructions.append(encode_sw(x12, x0, 0x100));           assembly.append('sw   x12, 256(x0)')
    instructions.append(encode_beq(x0, x0, 0));               assembly.append('beq  x0, x0, 0  # halt')

    return instructions, assembly

MACHINE_CODE, ASSEMBLY_LISTING = generate_unrolled_bubble_sort_program()
print(f'[OK] Program generated: {len(MACHINE_CODE)} instructions ({len(MACHINE_CODE)*4} bytes)')
print(f'  Compare-swap blocks: 496 × 6 instructions = 2976')
print(f'  Epilogue: 4 instructions')

[OK] Program generated: 2980 instructions (11920 bytes)
  Compare-swap blocks: 496 × 6 instructions = 2976
  Epilogue: 4 instructions


## 4. Load Program into Instruction RAM

In [4]:
def upload_program(instructions: List[int]):
    """Write program to instruction memory.
    
    Note: In button controller mode, hold BTN0 before calling this
    to keep processor in reset during program load.
    """
    for idx, instr in enumerate(instructions):
        instruction_memory.write(idx * 4, instr & 0xFFFF_FFFF)
    print(f'[OK] Loaded {len(instructions)} instructions to IRAM')

# Initial program load (make sure BTN0 is held if processor is running!)
print('Initial program load...')
print('[WARNING] If processor is running,hold BTN0 now to prevent interference')
time.sleep(1)  # Give time to press button if needed

upload_program(MACHINE_CODE)

# Readback verification
print('  Readback check (first 4):')
for i in range(4):
    expected = MACHINE_CODE[i] & 0xFFFF_FFFF
    actual = instruction_memory.read(i * 4)
    status = 'PASS' if actual == expected else 'FAIL'
    print(f'    [{i}] 0x{expected:08X} -> 0x{actual:08X} {status}')

Initial program load...
[WARNING] If processor is running,hold BTN0 now to prevent interference
[OK] Loaded 2980 instructions to IRAM
  Readback check (first 4):
    [0] 0x00002383 -> 0x00002383 PASS
    [1] 0x00402403 -> 0x00402403 PASS
    [2] 0x007424B3 -> 0x007424B3 PASS
    [3] 0x00048663 -> 0x00048663 PASS


## 5. Helper Functions

In [5]:
# ── Type conversions ─────────────────────────────────────────────────────────
def signed_to_unsigned(val: int) -> int:
    return struct.unpack('<I', struct.pack('<i', val))[0]

def unsigned_to_signed(val: int) -> int:
    return struct.unpack('<i', struct.pack('<I', val & 0xFFFF_FFFF))[0]

# ── Memory I/O ───────────────────────────────────────────────────────────────
def write_array_to_dram(arr: List[int], base: int = DRAM_SORT_ARRAY_BASE_ADDR):
    for i, val in enumerate(arr):
        data_memory.write(base + i * 4, signed_to_unsigned(val))

def read_array_from_dram(size: int, base: int = DRAM_SORT_ARRAY_BASE_ADDR) -> List[int]:
    return [unsigned_to_signed(data_memory.read(base + i * 4)) for i in range(size)]

def clear_completion_flag():
    data_memory.write(DRAM_COMPLETION_FLAG_ADDR, 0x00000000)

def read_completion_flag() -> int:
    return data_memory.read(DRAM_COMPLETION_FLAG_ADDR) & 0xFFFF_FFFF

# ── Test vector generation ───────────────────────────────────────────────────
def generate_test_array(size: int = NUM_ELEMENTS_TO_SORT, seed: int = PRNG_SEED_VALUE) -> List[int]:
    rng = random.Random(seed)
    arr = [rng.randint(SIGNED_INT32_MINIMUM_VALUE, SIGNED_INT32_MAXIMUM_VALUE) for _ in range(size)]
    arr[0] = SIGNED_INT32_MINIMUM_VALUE
    arr[1] = SIGNED_INT32_MAXIMUM_VALUE
    arr[2] = 0
    arr[3] = -1
    return arr

print('[OK] Helper functions defined')

[OK] Helper functions defined


## 6. Physical Button Testing (Button Controller Bitstream)

**Prerequisites:** Bitstream must include `button_controller` module connected to processor

This notebook uses **physical buttons on the PYNQ board** to control execution.

### Button Functions:
- **BTN0**: Reset button - Press to hold processor in reset, release to run
- **BTN1**: Halt/Run toggle
- **BTN2**: Single-step mode
- **BTN3**: Force done flag

### LED Status:
- **LED0 (R14)**: `done_flag_0` from processor (completion indicator)
- **LED1-LED3**: Button controller status

In [6]:
# ══════════════════════════════════════════════════════════════════════════════
# PHYSICAL BUTTON TESTING WORKFLOW
# ══════════════════════════════════════════════════════════════════════════════

def is_processor_in_reset() -> bool:
    """
    Detect if processor is in reset by checking if it's executing.
    
    Method: Write a sentinel value, wait briefly, check if processor overwrote it.
    - If value unchanged: processor is in reset (or halted)
    - If value changed to 0xDEADBEAF: processor is running and completed
    
    Returns:
        True if processor appears to be in reset/halted
        False if processor is actively running
    """
    # Write a test sentinel (different from completion value)
    TEST_SENTINEL = 0xCAFEBABE
    data_memory.write(DRAM_COMPLETION_FLAG_ADDR, TEST_SENTINEL)
    time.sleep(0.2)  # Give processor time to run if it's active
    
    current = read_completion_flag()
    if current == TEST_SENTINEL:
        return True  # Value unchanged - processor not running
    else:
        return False  # Value changed - processor is active

def wait_for_button_press(timeout: float = 10.0) -> bool:
    """
    Wait for user to press BTN0 (processor enters reset).
    Shows live status and countdown.
    
    Returns:
        True if button pressed within timeout
        False if timeout reached
    """
    print('      [PRESS] Press and HOLD BTN0 now!')
    print('      (Watching for processor to enter reset...)')
    
    start = time.perf_counter()
    while time.perf_counter() - start < timeout:
        elapsed = time.perf_counter() - start
        remaining = timeout - elapsed
        
        in_reset = is_processor_in_reset()
        status = '[IN RESET]' if in_reset else '[RUNNING]'
        
        print(f'\r      [{remaining:.1f}s] Status: {status}   ', end='', flush=True)
        
        if in_reset:
            print('\n      [OK] Button pressed - processor in reset!')
            return True
        
        time.sleep(0.2)
    
    print('\n      [TIMEOUT] Button not detected')
    return False

def wait_for_button_release(timeout: float = 10.0) -> bool:
    """
    Wait for user to release BTN0 (processor starts running).
    Shows live status and countdown.
    
    Returns:
        True if button released within timeout
        False if timeout reached
    """
    print('      [RELEASE] RELEASE BTN0 now to start execution!')
    print('      (Watching for processor to start running...)')
    
    start = time.perf_counter()
    last_state = is_processor_in_reset()
    
    while time.perf_counter() - start < timeout:
        elapsed = time.perf_counter() - start
        remaining = timeout - elapsed
        
        in_reset = is_processor_in_reset()
        status = '[RESET]' if in_reset else '[RUNNING]'
        
        print(f'\r      [{remaining:.1f}s] Status: {status}   ', end='', flush=True)
        
        # Detect transition from reset to running
        if last_state and not in_reset:
            print('\n      [OK] Button released - processor started!')
            return True
        
        last_state = in_reset
        time.sleep(0.2)
    
    print('\n      [TIMEOUT] Button release not detected')
    return False

def test_with_physical_buttons(test_array: List[int], poll_interval: float = 0.1):
    """
    Load data and wait for physical button to trigger execution.
    
    IMPORTANT: This assumes button controller bitstream (NOT GPIO control).
    Automatically detects button press/release state.
    
    User workflow:
    1. Press BTN0 when prompted (auto-detected)
    2. Python loads program/data while BTN0 held
    3. Release BTN0 when prompted (auto-detected)
    4. Python monitors done_flag_0 for completion
    
    Args:
        test_array: Array to sort
        poll_interval: How often to check done_flag (seconds)
    """
    print('=' * 70)
    print('PHYSICAL BUTTON TESTING MODE')
    print('=' * 70)
    
    # Step 1: Wait for button press (auto-detect)
    print('\n[1/4] Waiting for BTN0 press...')
    if not wait_for_button_press(timeout=10.0):
        print('      [WARNING] Continuing anyway - make sure BTN0 is held!')
        time.sleep(2)
    
    # Step 2: Load program and data while button is held
    print('\n[2/4] Loading program and test data (keep BTN0 held)...')
    
    # Clear completion flag
    for _ in range(3):
        data_memory.write(DRAM_COMPLETION_FLAG_ADDR, 0x00000000)
        time.sleep(0.01)
    
    # Verify cleared
    current_flag = read_completion_flag()
    print(f'      [OK] Completion flag cleared: 0x{current_flag:08X}')
    
    # Load program
    for idx, instr in enumerate(MACHINE_CODE):
        instruction_memory.write(idx * 4, instr & 0xFFFF_FFFF)
    print(f'      [OK] Loaded {len(MACHINE_CODE)} instructions to IRAM')
    
    # Load test data
    write_array_to_dram(test_array)
    print(f'      [OK] {len(test_array)} elements loaded')
    print(f'      Input array ({len(test_array)} elements):')
    print(f'      {test_array}')
    
    # Step 3: Wait for button release (auto-detect)
    print('\n[3/4] Waiting for BTN0 release...')
    if not wait_for_button_release(timeout=10.0):
        print('      [WARNING] Timeout - monitoring anyway...')
    
    # Step 4: Monitor for completion
    print('\n      Monitoring for completion...')
    start_time = time.perf_counter()
    timeout = start_time + MAX_EXECUTION_TIMEOUT_SECONDS
    
    print('      ', end='', flush=True)
    while time.perf_counter() < timeout:
        flag = read_completion_flag()
        elapsed = time.perf_counter() - start_time
        
        # Show live status (update in place)
        print(f'\r      Status: 0x{flag:08X} | Elapsed: {elapsed:.2f}s', end='', flush=True)
        
        if flag == COMPLETION_MAGIC_VALUE:
            print()  # New line after status
            print(f'      [OK] Completion detected! ({elapsed:.3f}s)')
            break
        
        time.sleep(poll_interval)
    else:
        print()  # New line after status
        print(f'      [TIMEOUT] {MAX_EXECUTION_TIMEOUT_SECONDS}s exceeded')
        return False
    
    # Step 5: Verify results  
    print('\n[4/4] Verifying results...')
    result = read_array_from_dram(len(test_array))
    expected = sorted(test_array)
    
    passed = (result == expected)
    
    print(f'      Output array ({len(result)} elements):')
    print(f'      {result}')
    print()
    
    if passed:
        print('      [PASS] Output matches expected sorted order')
    else:
        print('      [FAIL] Output does NOT match expected sorted order')
        print()
        print(f'      Expected (sorted):')
        print(f'      {expected}')
        print()
        
        # Show detailed mismatch information
        mismatches = []
        for i in range(len(result)):
            if result[i] != expected[i]:
                mismatches.append((i, expected[i], result[i]))
        
        print(f'      [MISMATCH] Found {len(mismatches)} error(s):')
        print('      ' + '─' * 66)
        print(f'      {"Index":<8} {"Expected":<20} {"Got":<20} {"Diff":<16}')
        print('      ' + '─' * 66)
        for idx, exp, got in mismatches:
            diff = got - exp
            print(f'      [{idx:<6}] {exp:<20} {got:<20} ({diff:+d})')
        print('      ' + '─' * 66)
    
    print('=' * 70)
    return passed

print('[OK] Physical button testing function defined')
print()
print('Button state auto-detection enabled!')
print('The script will automatically detect when you press/release BTN0.')
print()
print('To run a test:')
print('  test_array = generate_test_array()')
print('  test_with_physical_buttons(test_array)')
print()
print('To manually check reset state:')
print('  is_processor_in_reset()  # Returns True if in reset, False if running')

[OK] Physical button testing function defined

Button state auto-detection enabled!
The script will automatically detect when you press/release BTN0.

To run a test:
  test_array = generate_test_array()
  test_with_physical_buttons(test_array)

To manually check reset state:
  is_processor_in_reset()  # Returns True if in reset, False if running


In [8]:
# ═══ Run Physical Button Test (with automatic flag clear) ═══════════════════
# This cell runs a complete test cycle

# Generate test data
test_array = generate_test_array(seed=42)

# Clear flag and wait for button press
test_with_physical_buttons(test_array)

PHYSICAL BUTTON TESTING MODE

[1/4] Waiting for BTN0 press...
      [PRESS] Press and HOLD BTN0 now!
      (Watching for processor to enter reset...)
      [6.8s] Status: [IN RESET]   
      [OK] Button pressed - processor in reset!

[2/4] Loading program and test data (keep BTN0 held)...
      [OK] Completion flag cleared: 0x00000000
      [OK] Loaded 2980 instructions to IRAM
      [OK] 32 elements loaded
      Input array (32 elements):
      [-100, 100, 0, -1, -30, -38, -43, -65, 88, -74, 73, 89, 39, -78, 51, 8, -92, -93, -77, -45, -41, 29, 54, -94, 43, -50, 83, 66, 79, 39, 7, -44]

[3/4] Waiting for BTN0 release...
      [RELEASE] RELEASE BTN0 now to start execution!
      (Watching for processor to start running...)
      [3.7s] Status: [RUNNING]   
      [OK] Button released - processor started!

      Monitoring for completion...
      Status: 0xDEADBEAF | Elapsed: 0.01s
      [OK] Completion detected! (0.006s)

[4/4] Verifying results...
      Output array (32 elements):
     

True

## 7. Quick Manual Clear (Optional)

If you just want to clear the flag without running a full test:

In [9]:
# Just clear the completion flag (for manual testing)
clear_completion_flag()
current = read_completion_flag()
print(f'[OK] Completion flag cleared: 0x{current:08X}')
print('  Press BTN0 on the board when ready')

[OK] Completion flag cleared: 0xDEADBEAF
  Press BTN0 on the board when ready


## 8. Memory Inspection Utilities

In [10]:
def dump_dram(start_addr: int = 0, num_words: int = 32):
    """Display DRAM contents in hex and decimal."""
    print(f'DRAM Dump (0x{start_addr:03X} - 0x{start_addr + num_words*4:03X})')
    print('=' * 70)
    for i in range(num_words):
        addr = start_addr + i * 4
        raw = data_memory.read(addr)
        signed = unsigned_to_signed(raw)
        print(f'  [0x{addr:03X}]  0x{raw:08X}  ({signed:12d})')

def dump_iram(start_addr: int = 0, num_words: int = 10):
    """Display IRAM contents (instructions) in hex."""
    print(f'IRAM Dump (0x{start_addr:03X} - 0x{start_addr + num_words*4:03X})')
    print('=' * 70)
    for i in range(min(num_words, len(ASSEMBLY_LISTING))):
        addr = start_addr + i * 4
        instr = instruction_memory.read(addr)
        asm = ASSEMBLY_LISTING[i] if i < len(ASSEMBLY_LISTING) else '???'
        print(f'  [0x{addr:03X}]  0x{instr:08X}   # {asm}')

# Example usage:
# dump_dram(0x000, 8)      # First 8 elements
# dump_dram(0x100, 1)      # Completion flag
# dump_iram(0x000, 6)      # First compare-swap block

## 9. Summary

In [11]:
print('=' * 70)
print('[OK] Physical button verification notebook ready')
print('=' * 70)
print('Workflow:')
print('  1. Hold BTN0 (keeps processor in reset)')
print('  2. Run test cell (loads program/data)')
print('  3. Release BTN0 (starts execution)')
print('  4. Watch LED0 light up when complete')
print('  5. Results automatically verified')
print()
print('Manual operations:')
print('  - Clear flag: clear_completion_flag()')
print('  - Inspect memory: dump_dram(0x000, 32) or dump_dram(0x100, 1)')
print()
print('Press BTN0 to control processor reset')
print('=' * 70)

[OK] Physical button verification notebook ready
Workflow:
  1. Hold BTN0 (keeps processor in reset)
  2. Run test cell (loads program/data)
  3. Release BTN0 (starts execution)
  4. Watch LED0 light up when complete
  5. Results automatically verified

Manual operations:
  - Clear flag: clear_completion_flag()
  - Inspect memory: dump_dram(0x000, 32) or dump_dram(0x100, 1)

Press BTN0 to control processor reset
